In [2]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
if iskaggle:
    from pathlib import Path
    path = Path('/kaggle/input/imdb-dataset-of-50k-movie-reviews')
    ! pip install -q datasets

In [7]:
from pathlib import Path
path = Path('IMDB Dataset.csv')
import pandas as pd
df = pd.read_csv(path)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [8]:
df.describe(include='object')

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [9]:
df.loc[df['sentiment']=='positive','sentiment']=1
df.loc[df['sentiment']=='negative','sentiment']=0
df['sentiment'].apply(float)

0        1.0
1        1.0
2        1.0
3        0.0
4        1.0
        ... 
49995    1.0
49996    0.0
49997    0.0
49998    0.0
49999    0.0
Name: sentiment, Length: 50000, dtype: float64

In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

In [13]:
model_nm ='cardiffnlp/twitter-xlm-roberta-base-sentiment'


In [16]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm,model_max_length=256)
max_length = tokz.model_max_length

C:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marco\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [15]:
tokz.tokenize("G'day folks, I'm Jeremy from fast.ai!")

NameError: name 'tokz' is not defined

In [61]:
def tok_func(x): return tokz(x["review"],truncation=True,
  max_length=max_length)

In [73]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

In [63]:
tok_ds

Dataset({
    features: ['review', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 50000
})

In [64]:
tok_ds[5]['review'], tok_ds[5]['input_ids']


('Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.',
 [0,
  1250,
  17105,
  538,
  759,
  756,
  9,
  6032,
  40304,
  14277,
  4,
  10,
  13765,
  111,
  15970,
  9393,
  7432,
  4,
  209201,
  136,
  176762,
  191,
  47,
  10,
  110,
  2661,
  22304,
  4,
  1284,
  442,
  25,
  7,
  959,
  19542,
  7668,
  707,
  3017,
  214,
  5,
  1650,
  1660,
  8306,
  62163,
  10332,
  4,
  8,

In [65]:
tokz.vocab['.']

5

In [74]:
tok_ds = tok_ds.rename_columns({'sentiment':'labels'})
tok_ds

Dataset({
    features: ['review', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 50000
})

In [75]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['review', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 37500
    })
    test: Dataset({
        features: ['review', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 12500
    })
})

In [24]:
from transformers import TrainingArguments,Trainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [84]:
bs = 4
epochs = 1
lr = 8e-5

In [85]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [79]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred).float()}

In [86]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2,ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz,compute_metrics=corr_d)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
tok_ds = tok_ds.remove_columns('review')

In [81]:
tok_ds

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})

In [52]:
tokz.model_max_length

5000

In [87]:
trainer.train();

Epoch,Training Loss,Validation Loss


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 779865984 vs 779865872